In [79]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
import os

In [80]:
milb_batters_2021 = pd.read_csv("milb_data/milb_batter_2021.csv")
milb_batters_2022 = pd.read_csv("milb_data/milb_batter_2022.csv")
milb_batters_2023 = pd.read_csv("milb_data/milb_batter_2023.csv")
milb_pitchers_2021 = pd.read_csv("milb_data/milb_pitcher_2021.csv")
milb_pitchers_2022 = pd.read_csv("milb_data/milb_pitcher_2022.csv")
milb_pitchers_2023 = pd.read_csv("milb_data/milb_pitcher_2023.csv")

milb_batters = pd.concat([milb_batters_2021, milb_batters_2022, milb_batters_2023], ignore_index = True)
milb_pitchers = pd.concat([milb_pitchers_2021, milb_pitchers_2022, milb_pitchers_2023], ignore_index = True)

In [81]:
mlb_batters_2021 = pd.read_csv("mlb_data/mlb_batter_2021.csv")
mlb_batters_2022 = pd.read_csv("mlb_data/mlb_batter_2022.csv")
mlb_batters_2023 = pd.read_csv("mlb_data/mlb_batter_2023.csv")
mlb_batters_2024 = pd.read_csv("mlb_data/mlb_batter_2024.csv")
mlb_pitchers_2021 = pd.read_csv("mlb_data/mlb_pitcher_2021.csv")
mlb_pitchers_2022 = pd.read_csv("mlb_data/mlb_pitcher_2022.csv")
mlb_pitchers_2023 = pd.read_csv("mlb_data/mlb_pitcher_2023.csv")
mlb_pitchers_2024 = pd.read_csv("mlb_data/mlb_pitcher_2024.csv")

mlb_batters = pd.concat([mlb_batters_2021, mlb_batters_2022, mlb_batters_2023, mlb_batters_2024], ignore_index = True)
mlb_pitchers = pd.concat([mlb_pitchers_2021, mlb_pitchers_2022, mlb_pitchers_2023, mlb_pitchers_2024], ignore_index = True)

In [82]:
milb_pitchers = milb_pitchers.drop(columns = ['Pitches', 'Total', 'Pitch %', 'Hits', '1B', '2B', '3B', 'HR', 'SO', 'BB', 'Whiffs', 'Swings', 'Barrels', 'Dist (ft)', 'Spin (RPM)'])
mlb_pitchers = mlb_pitchers.drop(columns = ['Pitches', 'Total', 'Pitch %', 'Hits', '1B', '2B', '3B', 'HR', 'SO', 'BB', 'Whiffs', 'Swings', 'Barrels', 'Dist (ft)', 'Spin (RPM)'])
milb_batters = milb_batters.drop(columns = ['Pitches', 'Total', 'Pitch %', 'Hits', '1B', '2B', '3B', 'HR', 'SO', 'BB', 'Whiffs', 'Swings', 'Barrels', 'Dist (ft)'])
mlb_batters = mlb_batters.drop(columns = ['Pitches', 'Total', 'Pitch %', 'Hits', '1B', '2B', '3B', 'HR', 'SO', 'BB', 'Whiffs', 'Swings', 'Barrels', 'Dist (ft)'])

In [83]:
milb_batters['Player'] = milb_batters['Player'].str.replace(r' \w+ #\d+', '', regex = True)
milb_pitchers['Player'] = milb_pitchers['Player'].str.replace(r' \w+ #\d+', '', regex = True)
mlb_batters['Player'] = mlb_batters['Player'].str.replace(r' \w+ #\d+', '', regex = True)
mlb_pitchers['Player'] = mlb_pitchers['Player'].str.replace(r' \w+ #\d+', '', regex = True)

In [84]:
milb_pitchers = milb_pitchers.sort_values(by = ['Player', 'Year'])
milb_pitchers = milb_pitchers.drop_duplicates(subset = 'Player', keep = 'last')

mlb_pitchers = mlb_pitchers.sort_values(by = ['Player', 'Year'])

milb_batters = milb_batters.sort_values(by = ['Player', 'Year'])
milb_batters = milb_batters.drop_duplicates(subset = 'Player', keep = 'last')

mlb_batters = mlb_batters.sort_values(by = ['Player', 'Year'])

In [85]:
common_pitchers = set(milb_pitchers['Player']).intersection(mlb_pitchers['Player'])
milb_pitchers = milb_pitchers[milb_pitchers['Player'].isin(common_pitchers)]
mlb_pitchers = mlb_pitchers[mlb_pitchers['Player'].isin(common_pitchers)]

common_batters = set(milb_batters['Player']).intersection(mlb_batters['Player'])
milb_batters = milb_batters[milb_batters['Player'].isin(common_batters)]
mlb_batters = mlb_batters[mlb_batters['Player'].isin(common_batters)]


In [86]:
def combine_leagues(milb, mlb, stats_columns):
    for year_offset in range(3):
        for stat in stats_columns:
            milb[f'{stat}_{year_offset}'] = pd.NA
    
    for idx, milb_row in milb.iterrows():
        player = milb_row['Player']
        milb_year = milb_row['Year']
        
        player_mlb_data = mlb[mlb['Player'] == player]
        
        
        for year_offset in range(3):
            target_year = milb_year + year_offset
            matching_mlb = player_mlb_data[player_mlb_data['Year'] == target_year]
            
            if not matching_mlb.empty:
                for stat in stats_columns:
                    milb.at[idx, f'{stat}_{year_offset}'] = matching_mlb[stat].values[0]
    
    return milb

batter_columns = ['K%', 'BB%', 'BA', 'xBA', 'OBP', 'xOBP',
       'SLG', 'xSLG', 'wOBA', 'xwOBA', 'BABIP', 'ISO', 'Whiff%', 'EV (MPH)',
       'Adj. EV (MPH)', 'LA (°)', 'Hard Hit%', 'Barrel/BBE%', 'Barrel/PA%']

pitcher_columns = ['K%', 'BB%', 'BA', 'xBA', 'OBP', 'xOBP',
       'SLG', 'xSLG', 'wOBA', 'xwOBA', 'BABIP', 'ISO', 'Whiff%', 'Pitch (MPH)',
       'Perceived Velocity', 'Extension (ft)', 'PX (ft)', 'PZ (ft)',
       'EV (MPH)', 'Adj. EV (MPH)', 'LA (°)', 'Hard Hit%', 'Barrel/BBE%',
       'Barrel/PA%']

milb_batters = combine_leagues(milb_batters, mlb_batters, batter_columns)
milb_pitchers = combine_leagues(milb_pitchers, mlb_pitchers, pitcher_columns)

In [87]:
def train_batter(milb_batters, batter_columns, year_plus_num_columns):
    milb_batters = milb_batters.dropna(subset=batter_columns + year_plus_num_columns)
    X = milb_batters[batter_columns]
    y = milb_batters[year_plus_num_columns]
    
    model = LinearRegression()
    model.fit(X, y)

    return model

def train_pitcher(milb_pitchers, pitcher_columns, year_plus_num_columns):
    milb_pitchers = milb_pitchers.dropna(subset=pitcher_columns + year_plus_num_columns)
    X = milb_pitchers[pitcher_columns]
    y = milb_pitchers[year_plus_num_columns]
    
    model = LinearRegression()
    model.fit(X, y)

    return model

year_plus_0_pitcher_columns = [col + '_0' for col in pitcher_columns]
model = train_pitcher(milb_pitchers, pitcher_columns, year_plus_0_pitcher_columns)
pitchers = pd.read_csv(f'milb_data/milb_pitcher_2024.csv')
pitcher = pitchers[:1]
pitcher_name = pitcher['Player'].iloc[0]
prediction = model.predict(pitcher[pitcher_columns])
prediction = pd.DataFrame(prediction, columns = year_plus_0_pitcher_columns)
prediction.insert(0, 'Player', pitcher_name)

In [88]:
pitcher[['Player'] + pitcher_columns].head()

,Player,K%,BB%,BA,xBA,OBP,xOBP,SLG,xSLG,wOBA,...,Perceived Velocity,Extension (ft),PX (ft),PZ (ft),EV (MPH),Adj. EV (MPH),LA (°),Hard Hit%,Barrel/BBE%,Barrel/PA%
0,"Lawrence, Casey RHP",14.9,6.6,0.296,0.279,0.347,0.331,0.527,0.449,0.373,...,82.6,6.0,-0.21,2.03,86.9,91.6,11,33.6,7.2,5.6


In [89]:
prediction.head()

,Player,K%_0,BB%_0,BA_0,xBA_0,OBP_0,xOBP_0,SLG_0,xSLG_0,wOBA_0,...,Perceived Velocity_0,Extension (ft)_0,PX (ft)_0,PZ (ft)_0,EV (MPH)_0,Adj. EV (MPH)_0,LA (°)_0,Hard Hit%_0,Barrel/BBE%_0,Barrel/PA%_0
0,"Lawrence, Casey RHP",13.368399,6.373553,0.300249,0.296272,0.353285,0.351953,0.51319,0.490386,0.368859,...,83.298299,6.097758,-0.162771,2.054167,87.476212,92.159307,11.459353,37.309837,8.491905,6.638377
